In [6]:
#!/usr/bin/python

import sys
import tensorflow as tf
#import tensorflow.contrib.learn.python.learn as learn
import tflearn
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import six
from sklearn.metrics import roc_curve, roc_auc_score

"""
    The first argv is the kfold number. Second argv is the step number.
"""
k = 3
i = 0
j = 0
comData = np.load("../inData/3D-conv/comData.npy")
comClass = np.load("../inData//3D-conv/comClass.npy")

def processClassData(classData):
    """
    Process classData.
    
    Returns a one-hot array of shape [len(classData), 2].
    """
    # Convert label data to one-hot array
          
    classDataOH = np.zeros((len(classData),2))
    classDataOH[np.arange(len(classData)), classData] = 1
    
    return classDataOH

comData = comData[..., np.newaxis]
comClassOH = processClassData(comClass)

kfoldData = np.array_split(comData, k)
kfoldLabelsOH = np.array_split(comClassOH, k)
kfoldLabels = np.array_split(comClass, k)

spec, sens, auc = np.load("../outData/diagAUC-3d_conv8-16_sub40_k2_80epoch.npy")

# Does spec, sens, and auc exist?
try:
    spec
except NameError:
    spec = np.zeros(1)
try:
    sens
except NameError:
    sens = np.zeros(1)
try:
    auc
except NameError:
    auc = np.zeros(1)

subsamp = 40
winWidth = 10 # Must be a factor of data length (2000) and even.
steps = (2*(2000/subsamp)/winWidth)-1
if (2000/subsamp)%2 != 1:
    steps -= 1

sess = tf.InteractiveSession()
tf.reset_default_graph()
tflearn.initializations.normal()

# Input layer:
net = tflearn.layers.core.input_data(shape=[None, winWidth, 19, 17, 1])

# First layer:
net = tflearn.layers.conv.conv_3d(net, 8, [5,5,5],  activation="leaky_relu")
net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

# Second layer:
net = tflearn.layers.conv.conv_3d(net, 16, [5,5,5], activation="leaky_relu")
net = tflearn.layers.conv.max_pool_3d(net, 2, strides=2)

# Fully connected layer
net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2", weight_decay=0.001, activation="leaky_relu")
#net = tflearn.layers.core.fully_connected(net, 1024, regularizer="L2", weight_decay=0.001, activation="leaky_relu")

# Dropout layer:
net = tflearn.layers.core.dropout(net, keep_prob=0.5)

# Output layer:
net = tflearn.layers.core.fully_connected(net, 2, activation="softmax")

net = tflearn.layers.estimator.regression(net, optimizer='adam', learning_rate=0.0001, loss='categorical_crossentropy')

model = tflearn.DNN(net, tensorboard_verbose=0)

dummyData = np.reshape(np.concatenate(kfoldData[:i] + kfoldData[i+1:], axis=0), [-1, 2000, 19, 17, 1])
dummyData = dummyData[:,::subsamp]
dummyData = dummyData[:,int(0 + j*winWidth/2): int(winWidth + j*winWidth/2)]
print(dummyData[0,:].shape)
dummyLabels = np.reshape(np.concatenate(kfoldLabelsOH[:i] + kfoldLabelsOH[i+1:], axis=0), [-1, 2])
model.fit(dummyData, dummyLabels, batch_size=8, n_epoch=10, show_metric=True)

# Get roc curve data
predicted = np.array(model.predict(np.array(kfoldData[i])[:,::subsamp][:,int(0 + j*winWidth/2): int(winWidth + j*winWidth/2)]))
auc = np.append(auc, roc_auc_score(kfoldLabels[i], predicted[:,1]))

illTest = []
healthTest = []
for index, item in enumerate(kfoldLabels[i]):
    if item == 1:
        illTest.append(kfoldData[i][index])
    if item == 0:
        healthTest.append(kfoldData[i][index])

healthLabel = np.tile([1,0], (len(healthTest), 1))
illLabel = np.tile([0,1], (len(illTest), 1))

sens = np.append(sens, model.evaluate(np.array(healthTest)[:,::subsamp][:,int(0 + j*winWidth/2): int(winWidth + j*winWidth/2)], healthLabel)[0])
spec = np.append(spec, model.evaluate(np.array(illTest)[:,::subsamp][:,int(0 + j*winWidth/2): int(winWidth + j*winWidth/2)], illLabel)[0])

print(spec, sens, auc)
np.save("../outData/diagAUC-3d_conv8-16_sub40_k2_80epoch.npy", (spec, sens, auc))

Training Step: 339  | total loss: 0.63160 | time: 0.073s
| Adam | epoch: 010 | loss: 0.63160 - acc: 0.6690 -- iter: 264/266
Training Step: 340  | total loss: 0.62547 | time: 0.074s
| Adam | epoch: 010 | loss: 0.62547 - acc: 0.6646 -- iter: 266/266
--


In [7]:
print(auc, spec, sens)

[ 0.5718757   0.60451598  0.55913257] [ 0.50704223  0.59154928  0.52112675] [ 0.58730161  0.65079367  0.5714286 ]
